# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:

%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

In [0]:
import numpy as np
import pandas as pd


In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test,y_test) = cifar10.load_data()

In [8]:
x_train.shape

(50000, 32, 32, 3)

In [13]:
x_test.shape

(10000, 32, 32, 3)

In [10]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [11]:
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
def split (x,y):
  x_train1 = []
  y_train1 = []
  x_train2 = []
  y_train2 = []
  for i in range (0,x.shape[0]):
      if y[i] in range(0,5):
         x_train1.append(x[i])
         y_train1.append(y[i])
      else: 
         x_train2.append(x[i])
         y_train2.append(y[i])
  return x_train1,y_train1, x_train2, y_train2


In [0]:
x_train1, y_train1, x_train2, y_train2 =  split(x_train, y_train)

In [0]:
x_train1=np.array(x_train1)
y_train1=np.array(y_train1)
x_train2=np.array(x_train2)
y_train2=np.array(y_train2)

In [38]:
print (x_train1.shape , y_train1.shape , x_train2.shape, y_train2.shape)

(25000, 32, 32, 3) (25000, 1) (25000, 32, 32, 3) (25000, 1)


In [34]:
np.unique(y_train1)

array([0, 1, 2, 3, 4], dtype=uint8)

In [37]:
np.unique(y_train2)

array([5, 6, 7, 8, 9], dtype=uint8)

In [0]:
x_test1, y_test1, x_test2, y_test2 =  split(x_test, y_test)

In [0]:
x_test1=np.array(x_test1)
y_test1=np.array(y_test1)
x_test2=np.array(x_test2)
y_test2=np.array(y_test2)

In [40]:
print (x_test1.shape , y_test1.shape , x_test2.shape, y_test2.shape)

(5000, 32, 32, 3) (5000, 1) (5000, 32, 32, 3) (5000, 1)


In [41]:
np.unique(y_test1)

array([0, 1, 2, 3, 4])

In [42]:
np.unique(y_test2)

array([5, 6, 7, 8, 9])

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [45]:
y_train1[1]

array([1], dtype=uint8)

In [0]:
trainY1 = tf.keras.utils.to_categorical(y_train1, num_classes=5)
testY1 = tf.keras.utils.to_categorical(y_test1, num_classes=5)


In [48]:
trainY1[1]

array([0., 1., 0., 0., 0.], dtype=float32)

Normalizing output for y_train2 & y_test2 to make it in range of 0-4 

In [0]:
trainY2=y_train2-5

In [64]:
y_train2[1]

array([9], dtype=uint8)

In [65]:
trainY2[1]

array([4], dtype=uint8)

In [0]:
testY2 = y_test2-5

In [67]:
y_test2[1]

array([8])

In [68]:
testY2[1]

array([3])

In [0]:
trainY2 = tf.keras.utils.to_categorical(trainY2, num_classes=5)
testY2 = tf.keras.utils.to_categorical(testY2, num_classes=5)

In [70]:
trainY2[1]

array([0., 0., 0., 0., 1.], dtype=float32)

In [71]:
testY2[1]

array([0., 0., 0., 1., 0.], dtype=float32)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization, Flatten, Reshape, Dropout, MaxPool2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
clear_session()
model = Sequential()
model.add(BatchNormalization(input_shape= (32,32,3)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(GlobalAveragePooling2D())
model.add(Dense(5, activation='softmax'))

#Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [74]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        1

In [75]:
model.fit(x_train1, trainY1,
                    epochs=10,
                    batch_size= 32,
                    validation_data=(x_test1,testY1))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 121s 5ms/sample - loss: 0.9075 - accuracy: 0.6342 - val_loss: 0.7542 - val_accuracy: 0.7092
Epoch 2/10
25000/25000 [==============================] - 120s 5ms/sample - loss: 0.6506 - accuracy: 0.7527 - val_loss: 0.5992 - val_accuracy: 0.7688
Epoch 3/10
25000/25000 [==============================] - 121s 5ms/sample - loss: 0.5428 - accuracy: 0.7987 - val_loss: 0.5421 - val_accuracy: 0.8014
Epoch 4/10
25000/25000 [==============================] - 121s 5ms/sample - loss: 0.4727 - accuracy: 0.8238 - val_loss: 0.5113 - val_accuracy: 0.8130
Epoch 5/10
25000/25000 [==============================] - 120s 5ms/sample - loss: 0.4348 - accuracy: 0.8361 - val_loss: 0.4465 - val_accuracy: 0.8404
Epoch 6/10
25000/25000 [==============================] - 120s 5ms/sample - loss: 0.3918 - accuracy: 0.8542 - val_loss: 0.4402 - val_accuracy: 0.8388
Epoch 7/10
25000/25000 [===========================

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [76]:
np.size(model.layers)

13

Freezing the layers in pre trained model

In [0]:
#Set pre-trained model layers to not trainable except the last layer which is dense layer
for layer in model.layers[:-1]:
    layer.trainable = False

model.layers[np.size(model.layers)-1].trainable = True

In [96]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        1

In [97]:
model.output

<tf.Tensor 'dense/Identity:0' shape=(None, 5) dtype=float32>

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
x=model.output

In [0]:
#Add output layer
prediction = tf.keras.layers.Dense(5,activation='softmax')(x)

In [100]:
model.input

<tf.Tensor 'batch_normalization_input:0' shape=(None, 32, 32, 3) dtype=float32>

In [0]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [0]:
#Compile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [103]:
#How does our overall model looks
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_input (I [(None, 32, 32, 3)]       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64)        256   

In [104]:
#executing the final model after transfer learning 
final_model.fit(x_train2, trainY2,
                    epochs=10,
                    batch_size= 32,
                    validation_data=(x_test2,testY2))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 35s 1ms/sample - loss: 1.3080 - accuracy: 0.4830 - val_loss: 1.1880 - val_accuracy: 0.4832
Epoch 2/10
25000/25000 [==============================] - 36s 1ms/sample - loss: 1.1276 - accuracy: 0.4961 - val_loss: 1.0957 - val_accuracy: 0.4868
Epoch 3/10
25000/25000 [==============================] - 36s 1ms/sample - loss: 1.0630 - accuracy: 0.4998 - val_loss: 1.0598 - val_accuracy: 0.4976
Epoch 4/10
25000/25000 [==============================] - 36s 1ms/sample - loss: 1.0293 - accuracy: 0.5046 - val_loss: 1.0272 - val_accuracy: 0.4948
Epoch 5/10
25000/25000 [==============================] - 36s 1ms/sample - loss: 1.0061 - accuracy: 0.5032 - val_loss: 1.0102 - val_accuracy: 0.4902
Epoch 6/10
25000/25000 [==============================] - 36s 1ms/sample - loss: 0.9931 - accuracy: 0.5043 - val_loss: 0.9985 - val_accuracy: 0.4952
Epoch 7/10
25000/25000 [==============================] -

We see that with all the layers freezed except dense, we see very poor accuracy. To increase accuracy, we need to unfreeze few more layers.

To increase the accuracy, we would need to unfreeze few last layers to be trained. 

In [0]:
#Unfreezing all layers after layer# 6
for layer in model.layers[6:]:
    layer.trainable = True

In [106]:
#We will need to recompile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_input (I [(None, 32, 32, 3)]       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64)        256   

In [107]:
#executing the final model after transfer learning 
final_model.fit(x_train2, trainY2,
                    epochs=10,
                    batch_size= 32,
                    validation_data=(x_test2,testY2))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.8708 - accuracy: 0.5372 - val_loss: 0.8352 - val_accuracy: 0.5814
Epoch 2/10
25000/25000 [==============================] - 53s 2ms/sample - loss: 0.7952 - accuracy: 0.5724 - val_loss: 0.7828 - val_accuracy: 0.5860
Epoch 3/10
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.7514 - accuracy: 0.5991 - val_loss: 0.7631 - val_accuracy: 0.6002
Epoch 4/10
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.6556 - accuracy: 0.7704 - val_loss: 0.5847 - val_accuracy: 0.8576
Epoch 5/10
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.4820 - accuracy: 0.8886 - val_loss: 0.4757 - val_accuracy: 0.8824
Epoch 6/10
25000/25000 [==============================] - 53s 2ms/sample - loss: 0.3919 - accuracy: 0.9071 - val_loss: 0.4468 - val_accuracy: 0.8762
Epoch 7/10
25000/25000 [==============================] -

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [110]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [112]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [113]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [114]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

In [135]:
twenty_test.target

array([2, 2, 2, ..., 2, 2, 1])

In [136]:
twenty_test.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [137]:
twenty_test.data[0:5]

["From: brian@ucsd.edu (Brian Kantor)\nSubject: Re: HELP for Kidney Stones ..............\nOrganization: The Avant-Garde of the Now, Ltd.\nLines: 12\nNNTP-Posting-Host: ucsd.edu\n\nAs I recall from my bout with kidney stones, there isn't any\nmedication that can do anything about them except relieve the pain.\n\nEither they pass, or they have to be broken up with sound, or they have\nto be extracted surgically.\n\nWhen I was in, the X-ray tech happened to mention that she'd had kidney\nstones and children, and the childbirth hurt less.\n\nDemerol worked, although I nearly got arrested on my way home when I barfed\nall over the police car parked just outside the ER.\n\t- Brian\n",
 'From: rind@enterprise.bih.harvard.edu (David Rind)\nSubject: Re: Candida(yeast) Bloom, Fact or Fiction\nOrganization: Beth Israel Hospital, Harvard Medical School, Boston Mass., USA\nLines: 37\nNNTP-Posting-Host: enterprise.bih.harvard.edu\n\nIn article <1993Apr26.103242.1@vms.ocom.okstate.edu>\n banschbach@

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
vect= TfidfVectorizer()

In [0]:
dtm_train = vect.fit_transform(twenty_train.data)
dtm_test = vect.transform(twenty_test.data)

In [124]:
print('DTM shape for training data: ',dtm_train.shape)
print('DTM shape for testing data: ',dtm_test.shape)


DTM shape for training data:  (2257, 35788)
DTM shape for testing data:  (1502, 35788)


In [125]:
dtm_train

<2257x35788 sparse matrix of type '<class 'numpy.float64'>'
	with 365886 stored elements in Compressed Sparse Row format>

In [0]:
features = vect.get_feature_names()

In [130]:
np.size(features)

35788

In [131]:
#first 10 features
features[:10]

['00',
 '000',
 '0000',
 '0000001200',
 '000005102000',
 '0001',
 '000100255pixel',
 '00014',
 '000406',
 '0007']

In [132]:
#last 10 features
features[-10:]

['zvi',
 'zvonko',
 'zwart',
 'zyeh',
 'zyklon',
 'zyxel',
 'zz',
 'zzz',
 'ªl',
 'íålittin']

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [144]:
# use logistic regression with text column only
logreg = LogisticRegression()
logreg.fit(dtm_train, twenty_train.target)

y_pred_class_train = logreg.predict(dtm_train)
print('Training accuracy: ', metrics.accuracy_score(twenty_train.target, y_pred_class_train))

y_pred_class_test = logreg.predict(dtm_test)
print('Testing accuracy: ', metrics.accuracy_score(twenty_test.target, y_pred_class_test))

Training accuracy:  0.9920248116969429
Testing accuracy:  0.8974700399467377
